# Spline implementation of JAK2-STAT5 signaling pathway
In this notebook a practical example of the usage of AMICI spline functionalities is shown.
The model under consideration is the JAK2-STAT5 signaling pathway ([Swameye et al., 2003](https://doi.org/10.1073/pnas.0237333100)),
in which the dynamics of the system depend on a measured input function (the quantity `pEpoR` in the model).

Following the approach of ([Schelker et al., 2012](https://doi.org/10.1093/bioinformatics/bts393)), a continuous approximation of this input function is estimated together with the other parameters.
As in the original paper, we will use a spline with logarithmic parameterization in order to enforce the positivity constraint.

The model of the signaling pathway will be implemented in SBML using AMICI's spline annotations, experimental data integrated using the PEtab format and parameter estimation will be carried out using the [pyPESTO](https://pypesto.readthedocs.io/) library.

In [1]:
%pip install pypesto

Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install fides

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import math
import logging
import contextlib
import multiprocessing
import copy

import numpy as np
import sympy as sp
import pandas as pd
from matplotlib import pyplot as plt

import libsbml
import amici
import petab
import pypesto
import pypesto.petab

In [4]:
# Number of multi-starts for MAP estimation
n_starts = 150
# n_starts = 0 # when loading results

In [5]:
# Set default pypesto engine/optimizer
pypesto_optimizer = pypesto.optimize.FidesOptimizer(verbose=logging.WARNING)
pypesto_engine = pypesto.engine.MultiProcessEngine()

Engine set up to use up to 8 processes in total. The number was automatically determined and might not be appropriate on some systems.


In [6]:
# If running as a Github action, just do the minimal amount of work required to check whether the code is working
if os.getenv('GITHUB_ACTIONS') is not None:
    n_starts = 20
    pypesto_optimizer = pypesto.optimize.FidesOptimizer(verbose=logging.WARNING, options=dict(maxiter=10))
    pypesto_engine = pypesto.engine.SingleCoreEngine()

In [7]:
# A dictionary to store different approaches for a final comparison
all_results = {}

## Spline approximation with few nodes, using finite differences for the derivatives
As a first attempt, we fix a small amount of nodes, create new parameters for the values of the splines at the nodes and let AMICI compute the derivative at the nodes by using finite differences.

### Creating the PEtab model

In [8]:
# Problem name
name = "Swameye_PNAS2003_5nodes_FD"

First, we create a spline to represent the input function `pEpoR`, parametrized by its values at the nodes.
Since the value of the input function reaches its steady state by the end of the experiment, we extrapolate constantly after that (useful if we need to simulate the model after the last spline node).

In [9]:
# Create spline for pEpoR
nodes = [0, 5, 10, 20, 60]
values_at_nodes = [sp.Symbol(f"pEpoR_t{str(t).replace('.', '_dot_')}") for t in nodes] # new parameter symbols for spline values
spline = amici.splines.CubicHermiteSpline(
    sbml_id='pEpoR', # matches name of species in SBML model
    evaluate_at=amici.sbml_utils.amici_time_symbol, # the spline is evaluated at the current time
    nodes=nodes,
    values_at_nodes=values_at_nodes, # values at the nodes (in linear scale)
    extrapolate=(None, "constant"), # because steady state is reached
    bc="auto", # automatically determined from extrapolate (bc at right end will be 'zero_derivative')
    logarithmic_parametrization=True,
)

We can then add the spline to a skeleton SBML model based on the d2d implementation by (Schelker et al., 2012).
The skeleton SBML model defines a species `pEpoR` which interacts with the other species,
but has no reactions or rate rules of its own.
The code below creates an assignment rule for `pEpoR` using the spline formula, completing the model.
The parameters `pEpoR_t*` are automatically added to the SBML model too (using nominal values of `0.1` and declaring them to be constant).

In [10]:
# Add spline formula to SBML model
sbml_doc = libsbml.SBMLReader().readSBML(os.path.join('Swameye_PNAS2003', 'swameye2003_model.xml'))
sbml_model = sbml_doc.getModel()
spline.add_to_sbml_model(sbml_model, auto_add=True, y_nominal=0.1, y_constant=True)

A skeleton PEtab problem is provided, containing parameter bounds, observable definitions and experimental data.
Of particular relevance is the noise model used for the measurements of `pEpoR`, normal additive noise with standard deviation equal to `0.0274 + 0.1 * pEpoR`;
this is the same choice used in (Schelker et al., 2012), where it was estimated from experimental replicates.

However, the parameters associated to the spline are to be added too.
The code below defines parameter bounds for them according to the PEtab format and then creates a full PEtab problem integrating them together with the edited SBML file.
The condition, measurement and observable PEtab tables do not require additional modification and can be used as they are.

In [11]:
# Extra parameters associated to the spline
spline_parameters_df = pd.DataFrame(
    dict(parameterScale='log', lowerBound=0.001, upperBound=10, nominalValue=0.1, estimate=1),
    index=pd.Series(list(map(str, values_at_nodes)), name="parameterId"),
)

In [12]:
# Create PEtab problem
petab_problem = petab.Problem(
    sbml_model,
    condition_df=petab.conditions.get_condition_df(os.path.join('Swameye_PNAS2003', 'swameye2003_conditions.tsv')),
    measurement_df=petab.measurements.get_measurement_df(os.path.join('Swameye_PNAS2003', 'swameye2003_measurements.tsv')),
    parameter_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_parameters.tsv'), spline_parameters_df],
        petab.parameters.get_parameter_df
    ),
    observable_df=petab.observables.get_observable_df(os.path.join('Swameye_PNAS2003', 'swameye2003_observables.tsv')),
)

The resulting PEtab problem can be checked for errors and exported to disk if needed.

In [13]:
# Check whether PEtab model is valid
assert not petab.lint_problem(petab_problem)

In [14]:
# Save PEtab problem to disk
# import shutil
# shutil.rmtree(name, ignore_errors=True)
# os.mkdir(name)
# petab_problem.to_files_generic(prefix_path=name)

### Creating the pyPESTO problem
We can now create a pyPESTO problem directly from the PEtab problem.
Due to technical limitations in AMICI, currently the PEtab problem has to be "flattened" before it can be simulated from, but such operation is merely syntactical and thus does not change the essence of the model.

In [15]:
# Problem must be "flattened" to be used with AMICI
petab.core.flatten_timepoint_specific_output_overrides(petab_problem)

In [16]:
# Check whether simulation from the PEtab problem works
# import amici.petab_simulate
# simulator = amici.petab_simulate.PetabSimulator(petab_problem)
# simulator.simulate(noise=False)

In [17]:
# Import PEtab problem into pyPESTO
pypesto_problem = pypesto.petab.PetabImporter(petab_problem, model_name=name).create_problem()

Compiling amici model to folder /home/dweindl/src/AMICI-devel/python/examples/example_splines_swameye/amici_models/Swameye_PNAS2003_5nodes_FD.
2023-04-24 08:57:48.938 - amici.petab_import - INFO - Importing model ...
2023-04-24 08:57:48.940 - amici.petab_import - INFO - Validating PEtab problem ...
2023-04-24 08:57:49.760 - amici.petab_import - INFO - Model name is 'Swameye_PNAS2003_5nodes_FD'.
Writing model code to '/home/dweindl/src/AMICI-devel/python/examples/example_splines_swameye/amici_models/Swameye_PNAS2003_5nodes_FD'.
2023-04-24 08:57:49.764 - amici.petab_import - INFO - Species: 14
2023-04-24 08:57:49.767 - amici.petab_import - INFO - Global parameters: 9
2023-04-24 08:57:49.770 - amici.petab_import - INFO - Reactions: 13
2023-04-24 08:57:50.137 - amici.petab_import - INFO - Observables: 26
2023-04-24 08:57:50.142 - amici.petab_import - INFO - Sigmas: 26
2023-04-24 08:57:50.379 - amici.petab_import - DEBUG - Adding output parameters to model: ['scale_pSTAT5', 'scale_tSTAT5', 

2023-04-24 08:58:03.150 - amici.de_export - DEBUG - Finished computing Jrz                       +++ (2.15E-02s)
2023-04-24 08:58:03.170 - amici.de_export - DEBUG - Finished computing rz                        +++ (4.72E-04s)
2023-04-24 08:58:03.174 - amici.de_export - DEBUG - Finished writing Jrz.cpp                      ++ (5.92E-02s)
2023-04-24 08:58:03.225 - amici.de_export - DEBUG - Finished running smart_jacobian             ++++ (4.16E-04s)
2023-04-24 08:58:03.245 - amici.de_export - DEBUG - Finished simplifying dJrzdsigma             ++++ (3.67E-04s)
2023-04-24 08:58:03.249 - amici.de_export - DEBUG - Finished computing dJrzdsigma                +++ (3.96E-02s)
2023-04-24 08:58:03.255 - amici.de_export - DEBUG - Finished writing dJrzdsigma.cpp               ++ (6.04E-02s)
2023-04-24 08:58:03.302 - amici.de_export - DEBUG - Finished running smart_jacobian             ++++ (3.88E-04s)
2023-04-24 08:58:03.321 - amici.de_export - DEBUG - Finished simplifying dJrzdz                 

2023-04-24 08:58:05.698 - amici.de_export - DEBUG - Finished running smart_multiply             ++++ (9.14E-03s)
2023-04-24 08:58:05.739 - amici.de_export - DEBUG - Finished simplifying dydp                   ++++ (2.26E-02s)
2023-04-24 08:58:05.742 - amici.de_export - DEBUG - Finished computing dydp                      +++ (1.70E-01s)
2023-04-24 08:58:05.821 - amici.de_export - DEBUG - Finished writing dydp.cpp                     ++ (2.60E-01s)
2023-04-24 08:58:05.854 - amici.de_export - DEBUG - Finished computing dzdx                      +++ (4.82E-04s)
2023-04-24 08:58:05.857 - amici.de_export - DEBUG - Finished writing dzdx.cpp                     ++ (1.66E-02s)
2023-04-24 08:58:05.887 - amici.de_export - DEBUG - Finished computing dzdp                      +++ (5.03E-04s)
2023-04-24 08:58:05.890 - amici.de_export - DEBUG - Finished writing dzdp.cpp                     ++ (1.60E-02s)
2023-04-24 08:58:05.920 - amici.de_export - DEBUG - Finished computing drzdx                    

2023-04-24 08:58:07.580 - amici.de_export - DEBUG - Finished simplifying dx_rdatadtcl           ++++ (3.30E-04s)
2023-04-24 08:58:07.583 - amici.de_export - DEBUG - Finished computing dx_rdatadtcl              +++ (3.54E-02s)
2023-04-24 08:58:07.590 - amici.de_export - DEBUG - Finished writing dx_rdatadtcl.cpp             ++ (5.62E-02s)
2023-04-24 08:58:07.608 - amici.de_export - DEBUG - Finished writing z.cpp                        ++ (1.59E-04s)
2023-04-24 08:58:07.626 - amici.de_export - DEBUG - Finished writing rz.cpp                       ++ (1.64E-04s)
2023-04-24 08:58:07.756 - amici.de_export - DEBUG - Finished generating cpp code                   + (1.11E+01s)
2023-04-24 08:59:38.738 - amici.de_export - DEBUG - Finished compiling cpp code                    + (9.10E+01s)
2023-04-24 08:59:38.780 - amici.petab_import - INFO - Finished Importing PEtab model                 (1.10E+02s)


running AmiciInstall
running build_ext
-- The C compiler identification is GNU 12.2.0
-- The CXX compiler identification is GNU 12.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test -Wall
-- Performing Test -Wall - Failed
-- Performing Test -Wno-unused-function
-- Performing Test -Wno-unused-function - Failed
-- Performing Test -Wno-unused-variable
-- Performing Test -Wno-unused-variable - Failed
-- Performing Test -Wno-unused-but-set-variable
-- Performing Test -Wno-unused-but-set-variable - Failed
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found O

### Maximum Likelihood estimation
Using pyPESTO we can optimize for the parameter vector that maximizes the probability of observing the experimental data (maximum likelihood estimation).

A multistart method with local gradient-based optimization is used and the results of each multistart can be visualized in a waterfall plot. 

## Spline approximation with many nodes, using finite differences for the derivatives
Five nodes is arguably not enough to represent all plausible input choices. Increasing the number of nodes would give the spline more freedom and it can be done with minimal changes to the example above. However, more degrees of freedom mean more chance of overfitting. Thus, following (Schelker et al., 2012), we will add a regularization term consisting in the squared L2 norm of the spline's curvature, which promotes smoother and less oscillating functions. The value for the regularization strength $\lambda$ is chosen by comparing the sum of squared normalized residuals with its expected value, which can be computing by assuming it is roughly $\chi^2$-distributed.

### Creating the PEtab model

In [28]:
# Problem name
name = "Swameye_PNAS2003_15nodes_FD"

In [29]:
# Create spline for pEpoR
nodes = [0, 2.5, 5.0, 7.5, 10.0, 12.5, 15.0, 17.5, 20, 25, 30, 35, 40, 50, 60]
values_at_nodes = [sp.Symbol(f"pEpoR_t{str(t).replace('.', '_dot_')}") for t in nodes]
spline = amici.splines.CubicHermiteSpline(
    sbml_id='pEpoR',
    evaluate_at=amici.sbml_utils.amici_time_symbol,
    nodes=nodes,
    values_at_nodes=values_at_nodes,
    extrapolate=(None, "constant"),
    bc="auto",
    logarithmic_parametrization=True,
)

The regularization term can be easily computed by symbolic manipulation of the spline expression using AMICI and SymPy. Since it is very commonly used, we already provide a function for it in AMICI. Note: we regularize the curvature of the spline, which for positivity-enforcing spline is the logarithm of the function.

In order add the regularization term to the PEtab likelihood, a dummy observable has to be created.

In [30]:
# Compute L2 norm of the curvature of pEpoR
regularization = spline.squared_L2_norm_of_curvature()

In [31]:
# Add a parameter for regularization strength
reg_parameters_df = pd.DataFrame(
    dict(parameterScale='log10', lowerBound=1e-6, upperBound=1e6, nominalValue=1.0, estimate=0),
    index=pd.Series(['regularization_strength'], name="parameterId"),
)
# Encode regularization term as an additional observable
reg_observables_df = pd.DataFrame(
    dict(observableFormula=str(regularization).replace('**', '^'), observableTransformation='lin', noiseFormula='1/regularization_strength', noiseDistribution='normal'),
    index=pd.Series(['regularization'], name="observableId"),
)
# and correspoding measurement
reg_measurements_df = pd.DataFrame(
    dict(observableId='regularization', simulationConditionId='condition1', measurement=0, time=0, observableTransformation='lin'),
    index=pd.Series([0]),
)

In [32]:
# Add spline formula to SBML model
sbml_doc = libsbml.SBMLReader().readSBML(os.path.join('Swameye_PNAS2003', 'swameye2003_model.xml'))
sbml_model = sbml_doc.getModel()
spline.add_to_sbml_model(sbml_model, auto_add=True, y_nominal=0.1, y_constant=True)

In [33]:
# Extra parameters associated to the spline
spline_parameters_df = pd.DataFrame(
    dict(parameterScale='log', lowerBound=0.001, upperBound=10, nominalValue=0.1, estimate=1),
    index=pd.Series(list(map(str, values_at_nodes)), name="parameterId"),
)

In [34]:
# Create PEtab problem
petab_problem = petab.Problem(
    sbml_model,
    condition_df=petab.conditions.get_condition_df(os.path.join('Swameye_PNAS2003', 'swameye2003_conditions.tsv')),
    measurement_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_measurements.tsv'), reg_measurements_df],
        petab.measurements.get_measurement_df
    ).reset_index(drop=True),
    parameter_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_parameters.tsv'), spline_parameters_df, reg_parameters_df],
        petab.parameters.get_parameter_df
    ),
    observable_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_observables.tsv'), reg_observables_df],
        petab.observables.get_observable_df
    ),
)

In [35]:
# Check whether PEtab model is valid
assert not petab.lint_problem(petab_problem)

In [36]:
# Save PEtab problem to disk
# import shutil
# shutil.rmtree(name, ignore_errors=True)
# os.mkdir(name)
# petab_problem.to_files_generic(prefix_path=name)

### Creating the pyPESTO problem

In [37]:
# Problem must be "flattened" to be used with AMICI
petab.core.flatten_timepoint_specific_output_overrides(petab_problem)

In [38]:
# Check whether simulation from the PEtab problem works
# import amici.petab_simulate
# simulator = amici.petab_simulate.PetabSimulator(petab_problem)
# simulator.simulate(noise=False)

In [39]:
# Import PEtab problem into pyPESTO
pypesto_problem = pypesto.petab.PetabImporter(petab_problem, model_name=name).create_problem()

Compiling amici model to folder /home/dweindl/src/AMICI-devel/python/examples/example_splines_swameye/amici_models/Swameye_PNAS2003_15nodes_FD.
2023-04-24 09:10:54.803 - amici.petab_import - INFO - Importing model ...
2023-04-24 09:10:54.807 - amici.petab_import - INFO - Validating PEtab problem ...
2023-04-24 09:10:56.512 - amici.petab_import - INFO - Model name is 'Swameye_PNAS2003_15nodes_FD'.
Writing model code to '/home/dweindl/src/AMICI-devel/python/examples/example_splines_swameye/amici_models/Swameye_PNAS2003_15nodes_FD'.
2023-04-24 09:10:56.519 - amici.petab_import - INFO - Species: 14
2023-04-24 09:10:56.522 - amici.petab_import - INFO - Global parameters: 19
2023-04-24 09:10:56.528 - amici.petab_import - INFO - Reactions: 13
2023-04-24 09:10:56.557 - amici.sbml_import - DEBUG - Finished validating SBML                    ++ (7.15E-03s)
2023-04-24 09:10:56.570 - amici.sbml_import - DEBUG - Finished converting SBML local parameters   ++ (1.02E-04s)
2023-04-24 09:10:56.585 - am

2023-04-24 09:11:22.594 - amici.de_export - DEBUG - Finished running smart_jacobian             ++++ (2.14E-04s)
2023-04-24 09:11:22.609 - amici.de_export - DEBUG - Finished simplifying dJzdz                  ++++ (2.76E-04s)
2023-04-24 09:11:22.611 - amici.de_export - DEBUG - Finished computing dJzdz                     +++ (2.68E-02s)
2023-04-24 09:11:22.614 - amici.de_export - DEBUG - Finished writing dJzdz.cpp                    ++ (3.91E-02s)
2023-04-24 09:11:22.645 - amici.de_export - DEBUG - Finished simplifying Jrz                    ++++ (2.41E-04s)
2023-04-24 09:11:22.647 - amici.de_export - DEBUG - Finished computing Jrz                       +++ (1.08E-02s)
2023-04-24 09:11:22.661 - amici.de_export - DEBUG - Finished computing rz                        +++ (4.47E-04s)
2023-04-24 09:11:22.664 - amici.de_export - DEBUG - Finished writing Jrz.cpp                      ++ (3.51E-02s)
2023-04-24 09:11:22.697 - amici.de_export - DEBUG - Finished running smart_jacobian             

2023-04-24 09:11:24.962 - amici.de_export - DEBUG - Finished computing dydx                      +++ (3.16E-01s)
2023-04-24 09:11:24.987 - amici.de_export - DEBUG - Finished writing dydx.cpp                     ++ (3.54E-01s)
2023-04-24 09:11:30.053 - amici.de_export - DEBUG - Finished running smart_jacobian            +++++ (5.03E+00s)
2023-04-24 09:11:33.130 - amici.de_export - DEBUG - Finished simplifying dydp                  +++++ (3.06E+00s)
2023-04-24 09:11:33.132 - amici.de_export - DEBUG - Finished computing dydp                     ++++ (8.12E+00s)
2023-04-24 09:11:33.147 - amici.de_export - DEBUG - Finished running smart_multiply             ++++ (6.52E-03s)
2023-04-24 09:11:35.790 - amici.de_export - DEBUG - Finished simplifying dydp                   ++++ (2.63E+00s)
2023-04-24 09:11:35.792 - amici.de_export - DEBUG - Finished computing dydp                      +++ (1.08E+01s)
2023-04-24 09:11:36.450 - amici.de_export - DEBUG - Finished writing dydp.cpp                   

2023-04-24 09:11:37.770 - amici.de_export - DEBUG - Finished writing dx_rdatadx_solver.cpp        ++ (2.31E-02s)
2023-04-24 09:11:37.801 - amici.de_export - DEBUG - Finished simplifying dx_rdatadp             ++++ (9.79E-03s)
2023-04-24 09:11:37.802 - amici.de_export - DEBUG - Finished computing dx_rdatadp                +++ (1.78E-02s)
2023-04-24 09:11:37.807 - amici.de_export - DEBUG - Finished writing dx_rdatadp.cpp               ++ (2.81E-02s)
2023-04-24 09:11:37.828 - amici.de_export - DEBUG - Finished running smart_jacobian             ++++ (1.37E-04s)
2023-04-24 09:11:37.839 - amici.de_export - DEBUG - Finished simplifying dx_rdatadtcl           ++++ (1.91E-04s)
2023-04-24 09:11:37.840 - amici.de_export - DEBUG - Finished computing dx_rdatadtcl              +++ (1.81E-02s)
2023-04-24 09:11:37.842 - amici.de_export - DEBUG - Finished writing dx_rdatadtcl.cpp             ++ (2.55E-02s)
2023-04-24 09:11:37.851 - amici.de_export - DEBUG - Finished writing z.cpp                      

running AmiciInstall
running build_ext
-- The C compiler identification is GNU 12.2.0
-- The CXX compiler identification is GNU 12.2.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Performing Test -Wall
-- Performing Test -Wall - Failed
-- Performing Test -Wno-unused-function
-- Performing Test -Wno-unused-function - Failed
-- Performing Test -Wno-unused-variable
-- Performing Test -Wno-unused-variable - Failed
-- Performing Test -Wno-unused-but-set-variable
-- Performing Test -Wno-unused-but-set-variable - Failed
-- Found OpenMP_C: -fopenmp (found version "4.5") 
-- Found OpenMP_CXX: -fopenmp (found version "4.5") 
-- Found O

### Maximum Likelihood estimation
We will optimize the problem for different values of the regularization strength $\lambda$, then compute the sum of squared normalized residuals for each of the resulting parameter vectors. The one for which such a value is nearest to its expected value of $15$ (the number of observations from the input function) will be chosen as the final estimate.

## Spline approximation with few nodes, optimizing derivatives explicitly
An alternative way to achieve higher expressivity, while not increasing the number of nodes, is to optimize the derivatives of the spline at the nodes instead of computing them by finite differencing. The risk of overfitting is still present, so we will include regularization as in the above example.

### Creating the PEtab model

In [48]:
# Problem name
name = "Swameye_PNAS2003_5nodes"

We now need to create additional parameters for the spline derivatives too.

In [49]:
# Create spline for pEpoR
nodes = [0, 5, 10, 20, 60]
values_at_nodes = [sp.Symbol(f"pEpoR_t{str(t).replace('.', '_dot_')}") for t in nodes]
derivatives_at_nodes = [sp.Symbol(f"derivative_pEpoR_t{str(t).replace('.', '_dot_')}") for t in nodes[:-1]]
spline = amici.splines.CubicHermiteSpline(
    sbml_id='pEpoR',
    evaluate_at=amici.sbml_utils.amici_time_symbol,
    nodes=nodes,
    values_at_nodes=values_at_nodes,
    derivatives_at_nodes=derivatives_at_nodes + [0], # last value is zero because steady state is reached
    extrapolate=(None, "constant"),
    bc="auto",
    logarithmic_parametrization=True,
)

In [50]:
# Compute L2 norm of the curvature of pEpoR
regularization = spline.squared_L2_norm_of_curvature()

In [51]:
# Add a parameter for regularization strength
reg_parameters_df = pd.DataFrame(
    dict(parameterScale='log10', lowerBound=1e-6, upperBound=1e6, nominalValue=1.0, estimate=0),
    index=pd.Series(['regularization_strength'], name="parameterId"),
)
# Encode regularization term as an additional observable
reg_observables_df = pd.DataFrame(
    dict(observableFormula=str(regularization).replace('**', '^'), observableTransformation='lin', noiseFormula='1/regularization_strength', noiseDistribution='normal'),
    index=pd.Series(['regularization'], name="observableId"),
)
# and correspoding measurement
reg_measurements_df = pd.DataFrame(
    dict(observableId='regularization', simulationConditionId='condition1', measurement=0, time=0, observableTransformation='lin'),
    index=pd.Series([0]),
)

In [52]:
# Add spline formula to SBML model
sbml_doc = libsbml.SBMLReader().readSBML(os.path.join('Swameye_PNAS2003', 'swameye2003_model.xml'))
sbml_model = sbml_doc.getModel()
spline.add_to_sbml_model(sbml_model, auto_add=True, y_nominal=0.1, y_constant=True)

In [53]:
# Derivative parameters must be added separately
for p in derivatives_at_nodes:
    amici.sbml_utils.add_parameter(sbml_model, p, value=0.0, constant=True)

In [54]:
# Extra parameters associated to the spline
spline_parameters_df1 = pd.DataFrame(
    dict(parameterScale='log', lowerBound=0.001, upperBound=10, nominalValue=0.1, estimate=1),
    index=pd.Series(list(map(str, values_at_nodes)), name="parameterId"),
)
spline_parameters_df2 = pd.DataFrame(
    dict(parameterScale='lin', lowerBound=-0.666, upperBound=0.666, nominalValue=0.0, estimate=1),
    index=pd.Series(list(map(str, derivatives_at_nodes)), name="parameterId"),
)

In [55]:
# Create PEtab problem
petab_problem = petab.Problem(
    sbml_model,
    condition_df=petab.conditions.get_condition_df(os.path.join('Swameye_PNAS2003', 'swameye2003_conditions.tsv')),
    measurement_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_measurements.tsv'), reg_measurements_df],
        petab.measurements.get_measurement_df
    ).reset_index(drop=True),
    parameter_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_parameters.tsv'), spline_parameters_df1, spline_parameters_df2, reg_parameters_df],
        petab.parameters.get_parameter_df
    ),
    observable_df=petab.core.concat_tables(
        [os.path.join('Swameye_PNAS2003', 'swameye2003_observables.tsv'), reg_observables_df],
        petab.observables.get_observable_df
    ),
)

In [56]:
# Check whether PEtab model is valid
assert not petab.lint_problem(petab_problem)

In [57]:
# Save PEtab problem to disk
# import shutil
# shutil.rmtree(name, ignore_errors=True)
# os.mkdir(name)
# petab_problem.to_files_generic(prefix_path=name)

### Creating the pyPESTO problem

In [58]:
# Problem must be "flattened" to be used with AMICI
petab.core.flatten_timepoint_specific_output_overrides(petab_problem)

In [59]:
# Check whether simulation from the PEtab problem works
# import amici.petab_simulate
# simulator = amici.petab_simulate.PetabSimulator(petab_problem)
# simulator.simulate(noise=False)

In [60]:
# Import PEtab problem into pyPESTO
pypesto_problem = pypesto.petab.PetabImporter(petab_problem, model_name=name).create_problem()

### Maximum Likelihood estimation

## Bibliography
Schelker, M. et al. (2012). “Comprehensive estimation of input signals and dynamics in biochemical reaction networks”. In: Bioinformatics 28.18, pp. i529–i534. doi: [10.1093/bioinformatics/bts393](https://doi.org/10.1093/bioinformatics/bts393).

Swameye, I. et al. (2003). “Identification of nucleocytoplasmic cycling as a remote sensor in cellular signaling by databased modeling”. In: Proceedings of the National Academy of Sciences 100.3, pp. 1028–1033. doi: [10.1073/pnas.0237333100](https://doi.org/10.1073/pnas.0237333100).